1 - IMPORT MAIN LIBRARIES AND CREATE GLOBAL VARIABLES

In [1]:
import pandas as pd
import os
import requests

source_path = '..\\Source\\'
raw_path = '..\\Raw\\'
api_url = 'https://olinda.bcb.gov.br/olinda/servico/Informes_ListaTarifasPorInstituicaoFinanceira/versao/v1/odata/ListaTarifasPorInstituicaoFinanceira(PessoaFisicaOuJuridica=@PessoaFisicaOuJuridica,CNPJ=@CNPJ)?%40PessoaFisicaOuJuridica=%27J%27&%40CNPJ=%27REPLACECNPJ%27'

2 - EXTRACT AND LOAD CSV DATA - RAW LAYER

In [2]:
# Get complete path for all CSV files in "source" directory
filepaths = [source_path + file for file in os.listdir(source_path) if file.endswith('.csv')]

# Concat multiple CSV files in one data frame
df_raw_banks = pd.concat([pd.read_csv(filepath, delimiter=';', encoding='cp1252') for filepath in filepaths])
df_raw_banks.info()

# Load combined CSV to raw folder
df_raw_banks.to_csv(raw_path + 'banks_raw_data.csv', sep=',', encoding='cp1252')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 0 to 134
Data columns (total 15 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Ano                                              892 non-null    int64  
 1   Trimestre                                        892 non-null    object 
 2   Categoria                                        892 non-null    object 
 3   Tipo                                             892 non-null    object 
 4   CNPJ IF                                          892 non-null    object 
 5   Instituição financeira                           892 non-null    object 
 6   Índice                                           892 non-null    object 
 7   Quantidade de reclamações reguladas procedentes  892 non-null    int64  
 8   Quantidade de reclamações reguladas - outras     892 non-null    int64  
 9   Quantidade de reclamações não re

3 - EXTRACT AND LOAD API DATA - RAW LAYER

In [4]:
# Execute API for each bank extracted
responseList = []
bankList = df_raw_banks[df_raw_banks['CNPJ IF'].str.strip() != '']

for cnpj in bankList['CNPJ IF']:
    response = requests.get(api_url.replace('REPLACECNPJ', cnpj))
    json = response.json()
    df_response = pd.DataFrame(json['value'])
    df_response['CNPJ'] = cnpj
    responseList.append(df_response)

df_raw_taxes = pd.concat(bank_taxes for bank_taxes in responseList)
df_raw_taxes.to_csv(raw_path + 'taxes_raw_data.csv', sep=',', encoding='cp1252')
df_raw_taxes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9744 entries, 0 to 15
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CodigoServico  9744 non-null   object 
 1   Servico        9744 non-null   object 
 2   Unidade        9744 non-null   object 
 3   DataVigencia   9744 non-null   object 
 4   ValorMaximo    9744 non-null   float64
 5   TipoValor      9744 non-null   object 
 6   Periodicidade  9744 non-null   object 
 7   CNPJ           9744 non-null   object 
dtypes: float64(1), object(7)
memory usage: 685.1+ KB
